In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [121]:
from selenium import webdriver
from bs4 import BeautifulSoup

# Path to your web driver (e.g., ChromeDriver)
driver_path = '/path/to/chromedriver'

# Initialize Selenium WebDriver
driver = webdriver.Chrome(driver_path)

# Open the webpage
url = "https://yourstory.com/search?q=HDFC"
driver.get(url)

# Wait for the page to load completely
driver.implicitly_wait(5)

# Extract the page's source code after JavaScript execution
html = driver.page_source

# Parse with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Example: Extract divs with specific class
div_content = soup.find_all('div', class_='your-class-name')
for div in div_content:
    print(div.text)

# Close the browser
driver.quit()


AttributeError: 'str' object has no attribute 'capabilities'

In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to fetch and parse the data
def fetch_data(url):
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Check for request errors
       #     print(response)

        # Parse the HTML content
        
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup)
        
        # Example: Get the title of the page
        title = soup.title.string if soup.title else "No Title Found"
        print(title)
        
        # Example: Get the first paragraph or some other content
        paragraph = soup.find('p').get_text() if soup.find('p') else "No Paragraph Found"
        
        return {"url": url, "title": title, "paragraph": paragraph}
    except requests.exceptions.RequestException as e:
        return {"url": url, "error": str(e)}

# URLs to scrape
urls = [  "https://yourstory.com/search?q=HDFC", "https://backend.finshots.in/backend/search/?q=HDFC"]

# Fetch data from both URLs
for url in urls:
    data = fetch_data(url)
    print(f"Data from {data['url']}:")
    print(f"Title: {data['title']}")
    print(f"First Paragraph: {data['paragraph']}\n")

In [47]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

# Function to fetch and parse the data

def fetch_data():
    df=pd.DataFrame(columns=['title','post_url','published_date','article'])
    # URLs to scrape
    keywords=["HDFC","Tata Motars"]
    # Fetch data from both URLs
    data=[]

    for keyword in keywords:
        url = f"https://backend.finshots.in/backend/search/?q={keyword}"
        try:
            # Fetch the webpage content
            response = requests.get(url)
            response.raise_for_status()  # Check for request errors
            #print(response.text)

            # Parse the HTML content
            
            soup = BeautifulSoup(response.content, 'html.parser')
            soup=json.loads(response.content)
            matches=soup["matches"]
            #print(matches[0])

            for match in matches:
                # Example: Get the title of the page
                title = match['title']
                #print(title)
                
                # url = match['post_url']
                post_url=match['post_url']
                published_date=match['published_date']
                req = requests.get(url=post_url)
                soup = BeautifulSoup(req.content, 'lxml')
                # Example: Get the first paragraph or some other content
                paragraph = soup.find('div',{"class":"post-content"}).text.strip()#.get_text() 
                #paragraph=soup.text.strip()
                row=pd.DataFrame({ "title": [title],"post_url": [post_url],'published_date':[published_date], "article": [paragraph]})
                #print(row)
                df=df._append(row,ignore_index=True)
            data.append(df)
        except requests.exceptions.RequestException as e:
            return {"url": url, "error": str(e)}

    return data

data=fetch_data()

In [48]:
def clean_data(data):
    for i in range(0,len(data)):
        # Sorting by column "published_date"
        data[i]=data[i].sort_values(by=['published_date'], ascending=False)
        data[i]=data[i].iloc[0:5,:]
        #print(data[i])
    return data

clean_data(data)[0]

,title,post_url,published_date,article
6,HDFC is in trouble — Has the elephant forgotte...,https://finshots.in/markets/hdfc-is-in-trouble/,2024-01-19T18:38:03.109000+00:00,"In today's Finshots, we explain why HDFC Bank'..."
8,"45 years later, HDFC bids goodbye to Dalal Street",https://finshots.in/archive/45-years-later-hdf...,2023-07-14T01:30:00+00:00,"On 12th July at 3:30 pm, the stock ticker HDFC..."
5,"HDFC bids farewell to its ₹10,000 crore educat...",https://finshots.in/archive/hdfc-bids-farewell...,2023-06-22T01:30:00+00:00,"In today’s Finshots, we explain why internatio..."
7,3 investing lessons from HDFC’s Prashant Jain,https://finshots.in/markets/investing-lessons-...,2022-09-09T10:46:32+00:00,In today's Finshots we describe a few lessons ...
3,Weekly Wrapup - Understanding the HDFC Merger,https://finshots.in/archive/weekly-wrapup-unde...,2022-04-08T16:51:54.934000+00:00,"Before we get to today's story, a quick recap ..."


In [50]:
def generate_sentiment_score(data):
    import random
    def mock_sentiment_api(text):
        sentiment_score = random.uniform(0, 1)
        return sentiment_score 

    for i in range(0,len(data)):
        data[i]['sentiment_score'] = data[i].apply(mock_sentiment_api, axis=1)
    return data
generate_sentiment_score(data)[0]

,title,post_url,published_date,article,sentiment_score
6,HDFC is in trouble — Has the elephant forgotte...,https://finshots.in/markets/hdfc-is-in-trouble/,2024-01-19T18:38:03.109000+00:00,"In today's Finshots, we explain why HDFC Bank'...",0.159779
8,"45 years later, HDFC bids goodbye to Dalal Street",https://finshots.in/archive/45-years-later-hdf...,2023-07-14T01:30:00+00:00,"On 12th July at 3:30 pm, the stock ticker HDFC...",0.632368
5,"HDFC bids farewell to its ₹10,000 crore educat...",https://finshots.in/archive/hdfc-bids-farewell...,2023-06-22T01:30:00+00:00,"In today’s Finshots, we explain why internatio...",0.426931
7,3 investing lessons from HDFC’s Prashant Jain,https://finshots.in/markets/investing-lessons-...,2022-09-09T10:46:32+00:00,In today's Finshots we describe a few lessons ...,0.757981
3,Weekly Wrapup - Understanding the HDFC Merger,https://finshots.in/archive/weekly-wrapup-unde...,2022-04-08T16:51:54.934000+00:00,"Before we get to today's story, a quick recap ...",0.832144


In [54]:
keys=["dd","ee"]
b={"ddd":1,'dee':2}
for i in range(0,len(keys)):
    a=b[f"d{keys[i]}"]
    print(a)

1
2


In [ ]:
import requests
import json
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import hashlib




def fetch_articles():
    # Code to scrape articles from finshots.in for keywords HDFC and Tata Motors
    query = ["HDFC","Tata Motors"]

    article_df_final = pd.DataFrame(columns=['aid','source','type','title','url','publishDate','text','query'])
    for each_query in query:
        article_df = pd.DataFrame(columns=['aid','source','type','title','url','publishDate','text'])
        r = requests.get(url="https://backend.finshots.in/backend/search/?q="+each_query)
        out = json.loads(r.content)
        print(out)
        matches = out['matches']
        for match in matches:
            title = match['title']
            publishDate = datetime.datetime.strptime(match['published_date'].split('T')[0],"%Y-%m-%d").date().strftime('%Y-%m-%d')
            url = match['post_url']
            req = requests.get(url=url)
            #print(req.text)
            soup = BeautifulSoup(req.content, 'lxml')
            p_tags = soup.find_all('div',{"class":"post-content"})
            text = ""
            hash_obj = hashlib.sha256(url.encode('utf-8'))
            hex_hash = hash_obj.hexdigest()
            aid = str(hex_hash)
            for p in p_tags:
                text+=p.text
            article_df = pd.concat([article_df, pd.DataFrame([[aid,'FinShots','Article',title,url,publishDate,text,each_query]],columns=['aid','source','type','title','url','publishDate','text','query'])],ignore_index=True)
        # Taking first five records
        article_df.sort_values(by='publishDate', ascending=False, inplace=True)
        first_five_records = article_df.head(5)
        article_df_final = pd.concat([article_df_final,first_five_records],ignore_index=True)
    
    
    
    return article_df_final

fetch_articles().head()
